In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
traffic_file = "project_data.csv"

# Read Purchasing File and store into Pandas data frame
traffic_df = pd.read_csv(traffic_file)

traffic_df.head()

,ACC_ID,ROAD_NO,ROAD_NAME,LONGITUDE,LATITUDE,CRASH_DATE,CRASH_TIME,SEVERITY,EVENT_NATURE,EVENT_TYPE,TOTAL_BIKE_INVOLVED,TOTAL_TRUCK_INVOLVED,TOTAL_HEAVY_TRUCK_INVOLVED,TOTAL_MOTOR_CYCLE_INVOLVED,TOTAL_OTHER_VEHICLES_INVOLVED,TOTAL_PEDESTRIANS_INVOLVED
0,9227453,10355,Povah Rd,122.229302,-17.929427,1/1/16,200.0,PDO Minor,Hit Object,NaN,0,0,0,0,0,0
1,9227808,1020997,Abernethy Rd,115.987176,-31.963001,1/1/16,0.0,PDO Major,Hit Object,NaN,0,0,0,0,0,0
2,9237537,1080305,Ray Cl,116.019921,-32.214429,1/1/16,1910.0,Medical,Hit Pedestrian,Involving Pedestrian,0,0,0,0,1,1
3,9237543,1260018,Clydesdale St,115.861867,-32.007798,1/1/16,NaN,PDO Major,Non Collision,NaN,0,0,0,0,1,0
4,9237547,1070128,Warnbro Sound Av,115.759332,-32.335668,1/1/16,135.0,PDO Major,Hit Object,NaN,0,0,0,0,1,0


In [ ]:
round(traffic_df[["LONGITUDE","LATITUDE"]],1)

In [3]:
# converting date column to date month year 

traffic_df[["day", "month", "year"]] = traffic_df["CRASH_DATE"].str.split("/", expand = True)
traffic_df.info()
traffic_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136676 entries, 0 to 136675
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ACC_ID                         136676 non-null  int64  
 1   ROAD_NO                        136676 non-null  object 
 2   ROAD_NAME                      136676 non-null  object 
 3   LONGITUDE                      136676 non-null  float64
 4   LATITUDE                       136676 non-null  float64
 5   CRASH_DATE                     136676 non-null  object 
 6   CRASH_TIME                     136344 non-null  float64
 7   SEVERITY                       136676 non-null  object 
 8   EVENT_NATURE                   128771 non-null  object 
 9   EVENT_TYPE                     35188 non-null   object 
 10  TOTAL_BIKE_INVOLVED            136676 non-null  int64  
 11  TOTAL_TRUCK_INVOLVED           136676 non-null  int64  
 12  TOTAL_HEAVY_TRUCK_INVOLVED    

,ACC_ID,ROAD_NO,ROAD_NAME,LONGITUDE,LATITUDE,CRASH_DATE,CRASH_TIME,SEVERITY,EVENT_NATURE,EVENT_TYPE,TOTAL_BIKE_INVOLVED,TOTAL_TRUCK_INVOLVED,TOTAL_HEAVY_TRUCK_INVOLVED,TOTAL_MOTOR_CYCLE_INVOLVED,TOTAL_OTHER_VEHICLES_INVOLVED,TOTAL_PEDESTRIANS_INVOLVED,day,month,year
0,9227453,10355,Povah Rd,122.229302,-17.929427,1/1/16,200.0,PDO Minor,Hit Object,NaN,0,0,0,0,0,0,1,1,16
1,9227808,1020997,Abernethy Rd,115.987176,-31.963001,1/1/16,0.0,PDO Major,Hit Object,NaN,0,0,0,0,0,0,1,1,16
2,9237537,1080305,Ray Cl,116.019921,-32.214429,1/1/16,1910.0,Medical,Hit Pedestrian,Involving Pedestrian,0,0,0,0,1,1,1,1,16
3,9237543,1260018,Clydesdale St,115.861867,-32.007798,1/1/16,NaN,PDO Major,Non Collision,NaN,0,0,0,0,1,0,1,1,16
4,9237547,1070128,Warnbro Sound Av,115.759332,-32.335668,1/1/16,135.0,PDO Major,Hit Object,NaN,0,0,0,0,1,0,1,1,16


In [4]:
traffic_df["TOTAL_MOTOR_CYCLE_INVOLVED"].value_counts()
#traffic_df["TOTAL_BIKE_INVOLVED"].value_counts()
traffic_df["TOTAL_OTHER_VEHICLES_INVOLVED"].value_counts()
round(traffic_df["LATITUDE"],1).value_counts()

# traffic_df.groupby

-31.9    38071
-32.0    33877
-32.1    18964
-31.8    11004
-31.7     4446
         ...  
-19.2        1
-24.3        1
-24.0        1
-19.4        1
-16.1        1
Name: LATITUDE, Length: 194, dtype: int64

In [ ]:
## TO-DO/EXPLORE >> This is just to explore the pattern/relationship between variable to see what is happening (the overall picture) after that we can hypothesis/test these relationships and weather they have a significant impact on accidents 

# How to round/Lat-lon to narrow down traffic zones (at the moment round to 1)

# Explore the relationships between the roads/suburb (lat/lon) and: 

    # Yearly pattern of accidents 

    # Pattern of accident by season/months 

    # Patterns of accident by day/night # use bin function to separate in 3 hour intervals -- > 

## ^^ Try and do groupby.severity

# How the accidents are in relation to: 
    
    # Year (time series) 

    # Season/months 

    # Day/night 

#Vehicle type (motorcycle/truck/car - other vehicle) - this can also link to cause potentially

    # Day/night 

    # Season/month
    
    # Year 

# Idea, if we can incorporate a colour map 